In [3]:
import pandas as pd
import numpy as np
# import pyodbc
# from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)

In [ ]:
import sys
import os

project_path = r"C:\Users\Administrator\BDI-predator"

# add path to sys.path if not already added
if project_path not in sys.path:
    sys.path.append(project_path)

In [ ]:
conn = pyodbc.connect(
    'DRIVER={SQL Server};SERVER=10.197.51.166;DATABASE=tempDBPredator;UID=Predator_ML;PWD=predatorml'
)

In [ ]:
sampling_pct = 1

In [ ]:
query = f'''
DECLARE @sampling_percentage FLOAT = {sampling_pct};

WITH tsf_c03 AS (
    SELECT *
    FROM Transaction_Summary_Fraud
    WHERE Channel = 'C03'
)

, t_base AS (
    SELECT
        c03.*,
        tsf_c03.Confirmed
    FROM C03_Details AS c03
    LEFT JOIN tsf_c03
        ON c03.[Transaction Serial No] = tsf_c03.[Transaction Serial No]
    WHERE Confirmed = '1' OR Confirmed = '0'
)

, t_clean AS (
	SELECT
		[Transaction Serial No],
		[Product Indicator],
		[Release Number],
		Status,
		[Originator Code],
		[Responder Code],
		MTI,
		[Primary Bit Map],
		[Secondary Bit Map],
		PANNumber,
		[Processing Code],
		[Transaction Amount],
		[Card Billing Amount],
		[Transmission Date],
		STAN,
		[Transaction Datetime],
		[Expiration Date],
		[Settlement Date],
		[Capture Date],
		MCC,
		[Country Code],
		[POS Entry Mode],
		[POS Condition Code],
		[Auth ID Response Length],
		[Merchant Id],
		Track2,
		[Retrival Reference Number],
		[Auth ID Response],
		[Response Code],
		[Card Acceptor TerminalID],
		[Card Acceptor ID],
		[Terminal Owner],
		[Terminal CIty],
		[Terminal State],
		[Terminal Country],
		[Additional Data],
		[Currency Code],
		[Terminal Data],
		[POS Data],
		[Banknet Data],
		[Recv Institution ID],
		[Account Identification1],
		[Account Identification2],
		[Auth AgentID],
		[Settlement Record],
		[Batch and Shift Data],
		[Settlement Data],
		[Account Indicator],
		[Pre authorization],
		[ATM Additional Data],
		Confirmed
	FROM (
		SELECT *,
			ROW_NUMBER() OVER(
				PARTITION BY PANNumber
				ORDER BY CHECKSUM([Transaction Serial No])
			) AS rn,
			COUNT(*) OVER(
				PARTITION BY PANNumber
			) AS total
		FROM t_base
		WHERE Confirmed = '0'
	) AS sub
	WHERE rn <= total * (@sampling_percentage)
)

, t_fraud AS (
	SELECT
		[Transaction Serial No],
		[Product Indicator],
		[Release Number],
		Status,
		[Originator Code],
		[Responder Code],
		MTI,
		[Primary Bit Map],
		[Secondary Bit Map],
		PANNumber,
		[Processing Code],
		[Transaction Amount],
		[Card Billing Amount],
		[Transmission Date],
		STAN,
		[Transaction Datetime],
		[Expiration Date],
		[Settlement Date],
		[Capture Date],
		MCC,
		[Country Code],
		[POS Entry Mode],
		[POS Condition Code],
		[Auth ID Response Length],
		[Merchant Id],
		Track2,
		[Retrival Reference Number],
		[Auth ID Response],
		[Response Code],
		[Card Acceptor TerminalID],
		[Card Acceptor ID],
		[Terminal Owner],
		[Terminal CIty],
		[Terminal State],
		[Terminal Country],
		[Additional Data],
		[Currency Code],
		[Terminal Data],
		[POS Data],
		[Banknet Data],
		[Recv Institution ID],
		[Account Identification1],
		[Account Identification2],
		[Auth AgentID],
		[Settlement Record],
		[Batch and Shift Data],
		[Settlement Data],
		[Account Indicator],
		[Pre authorization],
		[ATM Additional Data],
		Confirmed
	FROM t_base
	WHERE Confirmed = '1'
)

, t_final AS (
	SELECT * FROM t_clean
	UNION ALL
	SELECT * FROM t_fraud
)

SELECT
	t_final.PANNumber
	, t_final.[Transaction Datetime]
	, t_final.[Product Indicator]
	, t_final.[Transaction Amount]
	, t_final.[Card Billing Amount]
	, t_final.MCC
	, t_final.[Country Code]
	, t_final.[POS Entry Mode]
	, t_final.[POS Condition Code]
	, t_final.[Merchant Id]
	, t_final.[Auth ID Response]
	, t_final.[Response Code]
	, t_final.[Card Acceptor TerminalID]
	, t_final.[Card Acceptor ID]
	, t_final.[Terminal Owner]
	, t_final.[Terminal CIty]
	, t_final.[Terminal State]
	, t_final.[Terminal Country]
	, t_final.[Additional Data]
	, t_final.[Currency Code]
	, t_final.[Terminal Data]
	, t_final.[POS Data]
	, t_final.[Settlement Data]
	, t_final.[Account Indicator]
	, t_final.[Pre authorization]
	, t_final.[ATM Additional Data]
	, tscf.[Customer Age]
	, tscf.[Branch Name]
	, tscf.[Product Code]
	, tscf.[Customer Average Income]
	, tscf.[Card Status]
	, tscf.Balance
	, tscf.[Age of Open Account Transaction]
	, tscf.[Age of Active Card Transaction]
	, tscf.[Age of Open Account Active Card]
	, tscf.[Transfer To BDI Staff]
	, tscf.[BDI Staff]
	, tscf.[Account Open Year]
	, DATEDIFF(DAY, TRY_CAST(tscf.[Card Opening Date] AS datetime), TRY_CAST(tscf.[Transaction Datetime] AS datetime)) AS age_of_card_open_to_transaction
	, DATEDIFF(DAY, TRY_CAST(tscf.[Registration Date E-Channel?] AS datetime), TRY_CAST(tscf.[Transaction Datetime] AS datetime)) AS age_of_regist_echannel_to_transaction
	, tscf.[SDB Past Due Date]
	, tscf.[Box Type Size]
	, tscf.[Box Amount]
	, tscf.[Is SDB Past Due]
	, tscf.[Flag Out Branch]
	, tscf.[Is TD Hold Amount]
	, tscf.[Is Program Hold Amount]
	, tscf.[Is Office Hour]
	, tscf.VALAS
	, tscf.[Transaction Load Time]
	, tscf.[Date Birth]
	, tscf.[Transaction Channel]
	, tscf.[Last Transaction Datetime]
	, tscf.[Last Transaction Amount]
	, tscf.[Dr Last Transaction Date]
	, tscf.[Dr Last Transaction Amount]
	, tscf.[Cr Last Transaction Date]
	, tscf.[Cr Last Transaction Amount]
	, tscf.[Total Transaction Amount Last 1 Day]
	, tscf.[Dr Total Transaction Amount Last 1 Day]
	, tscf.[Cr Total Transaction Amount Last 1 Day]
	, tscf.[Total Transaction Amount Last 7 Days]
	, tscf.[Dr Total Transaction Amount Last 7 Days]
	, tscf.[Cr Total Transaction Amount Last 7 Days]
	, tscf.[Total Transaction Amount Last 1 Month?]
	, tscf.[Dr Total Transaction Amount Last 1 Month]
	, tscf.[Cr Total Transaction Amount Last 1 Month]
	, tscf.[Average Transaction Amount Last 1 Day]
	, tscf.[Average Transaction Amount Last 7 Days]
	, tscf.[Average Transaction Amount Last 1 Month?]
	, tscf.[Total Transaction Amount L15Mi]
	, tscf.[High Risk Customer]
	, tscf.[Product Name]
	, tscf.[Previous Account Status]
	, tscf.[Remark 4 (Tabungan)]
	, tscf.HoldAmountTB
	, tscf.[Remark 4 (Deposito)]
	, tscf.[Hold Amount DP]
	, tscf.[Age of SDB Expiry Date Transaction Date]
	, tscf.[SDB Product Code]
	, tscf.[SDB Balance]
	, tscf.[Account Reactivated Date]
	, tscf.[CC Branch Code (Acc)]
	, tscf.[Branch Name (Acc)]
	, tscf.[CC Branch Code (Card)]
	, tscf.[Branch Name (Card)]
	, tscf.[CC Branch Code (Active Dormant)]
	, tscf.[Branch Name (Active Dormant)]
	, tscf.[Branch Name (Trx)]
	, tscf.[Is WhiteList?]
	, tscf.[Is Blacklist Account?]
	, tscf.[Is Blacklist Cust?]
	, tscf.[MNemonic Trx Desc]
	, tscf.[Product Code To]
	, tscf.[Access Code]
	, tscf.[SDB Box No ]
	, tscf.[SDB Status]
	, tscf.[SDB Last Payment Amount]
	, tscf.[Exclude Flag Txn Desc]
	, tscf.[Product Code (Deposito)]
	, tscf.[Is Merchant BDI?]
	, tscf.[Merchant Acc Name]
	, tscf.[Card Issuer Name]
	, tscf.[Total Trx Num L1D]
	, tscf.[Total Trx Num L7D]
	, tscf.[Is High Risk Country?]
	, tscf.[Is Blacklist Merchant?]
	, tscf.[Is WatchList Merchant?]
	, tscf.[Is High Risk MCC?]
	, tscf.[Customer Email]
	, tscf.[Credit Limit]
	, tscf.[Cash Credit Limit]
	, tscf.[Card Product]
	, tscf.[Customer No]
	, tscf.[Customer Sex]
	, tscf.[Is Blacklist Card No?]
	, tscf.[Is Valid Card?]
	, tscf.Currency
	, tscf.[POS Entry Mode]
	, tscf.[PIN Entry Capability]
	, tscf.[Is Card Expired?]
	, tscf.[Is 3DSecure?]
	, tscf.[Card Brand]
	, tscf.[Is WhiteList Merchant?]
	, tscf.[Is BDI Email?]
	, tscf.[Is WhiteList Card No?]
	, tscf.[Is WhiteList Account No?]
	, tscf.[Is Account BDI Employee?]
	, tscf.[Cod Access]
	, tscf.[Transaction Type]
	, tscf.[Total Transaction Amount L10Mi]
	, tscf.[Customer Type]
	, tscf.[Total Transaction Amount L5min]
	, tscf.TotalTrxAmountContactless
	, tscf.[Branch City]
	, tscf.[Branch Zip Code]
	, tscf.[Is Whitelist CardNo Open?]
	, tscf.DaysOfWhitelistCardNoOpen
	, tscf.[IsVAAccount?]
	, tscf.[Is Exclude Teminal Id?]
	, tscf.isBDICard
	, tscf.[Is Watchlist Card No?]
	, tscf.[Source Channel]
	, tscf.IsBlackListTID
	, tscf.IsFirstTrx3DS
	, tscf.[Card Data Input Capability]
	, tscf.CustomerNationality
	, tscf.[Total Transaction Amount Contactless per EOD]
	, tscf.IsTopUp
	, tscf.IsBillPayment
	, tscf.IsPaymentToVA
	, tscf.[Total Transaction Amount to Same EWallet L1D]
	, tscf.[Total Transaction Amount to Diff Wallet L1D]
	, tscf.[Total Transaction Amount to Same VA L1D]
	, tscf.[Total Transaction Amount to Diff VA L1D]
	, tscf.[Is High Risk Country Threshold Amount?]
	, tscf.[Is High Risk Currency?]
	, tscf.[Is High Risk Currency Threshold Amt?]
	, tscf.IsQRtrx
	, tscf.IsDCashTrx
	, tscf.IsFTTrx
	, tscf.IsDCashTrx1
	, tscf.IsFTTrx1
	, tscf.[CummTrxAmountQRIS 10Mins]
	, tscf.[CummTrxAmountDcash 15Mins]
	, tscf.[CummAmount Trf Last 10min]
	, tscf.[TrxQRIS Last 7D]
	, tscf.[TransactionTrf Last 7D]
	, tscf.[CummAmount Trf Last 20min]
	, tscf.[CountTrxQRIS2 10mins]
	, tscf.[CountTrxDcash 15Mins]
	, tscf.[CountTrxTrf2 10Mins]
	, tscf.[CountTrxVA Last 7D]
	, tscf.[CountTrxInqVA 15mins]
	, tscf.[Total Trx Num L15Mins]
	, tscf.[TOTAL AMOUNT LESS THAN 2 DAYS]
	, tscf.[Is Whitelist CardNo Previlage ?]
	, tscf.IsWhiteListTID
	, tscf.IsWatchListTID
	, tscf.[Time Difference between Current and Previous Transaction]
	, tscf.[Total Transaction Amount L30Mi]
	, tscf.IsBucket1WatchListCardNo
	, tscf.IsBucket1BlacklistCardNo
	, tscf.IsBucket1WhiteListCardNo
	, tscf.IsBucket2WhiteListMerchant
	, tscf.IsBucket2WatchListMerchant
	, tscf.IsBucket2BlacklistMerchant
	, tscf.CountTrx11PMto2AM
	, tscf.IsBucket3WhiteListMerchant
	, tscf.IsBucket3WatchListMerchant
	, tscf.IsBucket3WhiteListMID
	, tscf.IsBucket3WatchListMID
	, tscf.IsBucket3BlacklistMID
	, tscf.IsFacebookBlacklistMerchant
	, t_final.Confirmed
FROM t_final
LEFT JOIN Transaction_Summary_Calculations_Fraud tscf
	ON t_final.[Transaction Serial No] = tscf.[Transaction Serial No]
'''

In [ ]:
df_c03 = pd.read_sql(query, conn)

In [4]:
df_c03 = pd.read_csv("data/mock_data_join_c03_tscf.csv")

/tmp/ipykernel_291744/1933900030.py:1: DtypeWarning: Columns (13,28,33,53,112,120,128,139,144,160,177,178,179,182,183,184,185,186,187) have mixed types. Specify dtype option on import or set low_memory=False.
  df_c03 = pd.read_csv("data/mock_data_join_c03_tscf.csv")


In [5]:
df_c03

,Unnamed: 0,PANNumber,Transaction Serial No,Transaction Datetime,Product Indicator,Transaction Amount,Card Billing Amount,MCC,Country Code,POS Entry Mode,POS Condition Code,Merchant Id,Auth ID Response,Response Code,Card Acceptor TerminalID,Card Acceptor ID,Terminal Owner,Terminal CIty,Terminal State,Terminal Country,Additional Data,Currency Code,Terminal Data,POS Data,Settlement Data,Account Indicator,Pre authorization,ATM Additional Data,Customer Age,Branch Name,Product Code,Customer Average Income,Card Status,Balance,Age of Open Account Transaction,Age of Active Card Transaction,Age of Open Account Active Card,Transfer To BDI Staff,BDI Staff,Account Open Year,age_of_card_open_to_transaction,age_of_regist_echannel_to_transaction,SDB Past Due Date,Box Type Size,Box Amount,Is SDB Past Due,Flag Out Branch,Is TD Hold Amount,Is Program Hold Amount,Is Office Hour,VALAS,Transaction Load Time,Date Birth,Transaction Channel,Last Transaction Datetime,Last Transaction Amount,Dr Last Transaction Date,Dr Last Transaction Amount,Cr Last Transaction Date,Cr Last Transaction Amount,Total Transaction Amount Last 1 Day,Dr Total Transaction Amount Last 1 Day,Cr Total Transaction Amount Last 1 Day,Total Transaction Amount Last 7 Days,Dr Total Transaction Amount Last 7 Days,Cr Total Transaction Amount Last 7 Days,Total Transaction Amount Last 1 Month?,Dr Total Transaction Amount Last 1 Month,Cr Total Transaction Amount Last 1 Month,Average Transaction Amount Last 1 Day,Average Transaction Amount Last 7 Days,Average Transaction Amount Last 1 Month?,Total Transaction Amount L15Mi,High Risk Customer,Product Name,Previous Account Status,Remark 4 (Tabungan),HoldAmountTB,Remark 4 (Deposito),Hold Amount DP,Age of SDB Expiry Date Transaction Date,SDB Product Code,SDB Balance,Account Reactivated Date,CC Branch Code (Acc),Branch Name (Acc),CC Branch Code (Card),Branch Name (Card),CC Branch Code (Active Dormant),Branch Name (Active Dormant),Branch Name (Trx),Is WhiteList?,Is Blacklist Account?,Is Blacklist Cust?,MNemonic Trx Desc,Product Code To,Access Code,SDB Box No,SDB Status,SDB Last Payment Amount,Exclude Flag Txn Desc,Product Code (Deposito),Is Merchant BDI?,Merchant Acc Name,Card Issuer Name,Total Trx Num L1D,Total Trx Num L7D,Is High Risk Country?,Is Blacklist Merchant?,Is WatchList Merchant?,Is High Risk MCC?,Customer Email,Credit Limit,Cash Credit Limit,Card Product,Customer No,Customer Sex,Is Blacklist Card No?,Is Valid Card?,Currency,POS Entry Mode.1,PIN Entry Capability,Is Card Expired?,Is 3DSecure?,Card Brand,Is WhiteList Merchant?,Is BDI Email?,Is WhiteList Card No?,Is WhiteList Account No?,Is Account BDI Employee?,Cod Access,Transaction Type,Total Transaction Amount L10Mi,Customer Type,Total Transaction Amount L5min,TotalTrxAmountContactless,Branch City,Branch Zip Code,Is Whitelist CardNo Open?,DaysOfWhitelistCardNoOpen,IsVAAccount?,Is Exclude Teminal Id?,isBDICard,Is Watchlist Card No?,Source Channel,IsBlackListTID,IsFirstTrx3DS,Card Data Input Capability,CustomerNationality,Total Transaction Amount Contactless per EOD,IsTopUp,IsBillPayment,IsPaymentToVA,Total Transaction Amount to Same EWallet L1D,Total Transaction Amount to Diff Wallet L1D,Total Transaction Amount to Same VA L1D,Total Transaction Amount to Diff VA L1D,Is High Risk Country Threshold Amount?,Is High Risk Currency?,Is High Risk Currency Threshold Amt?,IsQRtrx,IsDCashTrx,IsFTTrx,IsDCashTrx1,IsFTTrx1,CummTrxAmountQRIS 10Mins,CummTrxAmountDcash 15Mins,CummAmount Trf Last 10min,TrxQRIS Last 7D,TransactionTrf Last 7D,CummAmount Trf Last 20min,CountTrxQRIS2 10mins,CountTrxDcash 15Mins,CountTrxTrf2 10Mins,CountTrxVA Last 7D,CountTrxInqVA 15mins,Total Trx Num L15Mins,TOTAL AMOUNT LESS THAN 2 DAYS,Is Whitelist CardNo Previlage ?,IsWhiteListTID,IsWatchListTID,Time Difference between Current and Previous Transaction,Total Transaction Amount L30Mi,IsBucket1WatchListCardNo,IsBucket1BlacklistCardNo,IsBucket1WhiteListCardNo,IsBucket2WhiteListMerchant,IsBucket2WatchListMerchant,IsBucket2Bl

In [14]:
df_c03['Terminal State'].value_counts()

Terminal State
000             11075
CA               4085
NY                426
00                243
FL                196
                ...  
WICHITA             1
NEW YORK            1
+18587615232        1
8573898506          1
 SURUL              1
Name: count, Length: 148, dtype: int64

In [ ]:
list(df_c03['Terminal Owner'].unique())
# need to be grouped by a bigger categories

['Microsoft',
 'Microsoft*Store',
 'WWW.PAPER.ID',
 'XDT*GELORA.ID',
 'AGODA.COM ROYAL PLAZ',
 'Agoda',
 'APPLE.COM/BILL',
 'FACEBK *H4YV46LVP2',
 'GO TAXI APP',
 'Lazada ID',
 'IMMORTAL LACES',
 'GOOGLE*WALLET TEMP',
 'Microsoft*Microsoft 36',
 'AUDIBLE',
 'AGODA.COM',
 'BLIBLI-BLI-DIGITAL-DES',
 'DANA QROFF * TQ043697',
 'DOKU*STARLINK NON3DSNO',
 'BLIBLI-3DS',
 'WOLFIER LLC',
 'AGODA.COM SAFA KOST',
 'FACEBK *PQKXH6QVP2',
 'INTERPARK         6808',
 'AGODA.COM KAPAL GARD',
 'GOOGLE *TEMPORARY HOLD',
 'ALIPAY',
 'SHOPEE.CO.ID',
 'TOKOPEDIA.COM',
 'TikTok',
 'Grab* 5-C6NTJAXVHFCXEN',
 'NAME-CHEAP.COM',
 'BLUE BIRD GROUP',
 'GOOGLE *Google Storage',
 'METAPAY*ADS',
 'PIPO_MDES_PH1',
 'Stripe',
 'Grab* A-6IK4TXQGWH5A',
 'Mama Rama',
 'GOOGLE *ADS4731841042',
 'PlayStation Network',
 'GOOGLE *ADS9392376970',
 'GOOGLE *ADS7356438249',
 'DISCORD',
 'AIRBNB * HMQWK4JYB5',
 'PAPER.ID',
 'GOOGLE *SERVICES',
 'PROTECHTRADER',
 'Grab* A-6IO7ARAWWHNW',
 'EXPEDIA 72861019890452',
 'FACEBK *ADS',


In [9]:
df_c03.columns[df_c03.isna().all()].tolist()

['Customer Average Income',
 'Transfer To BDI Staff',
 'BDI Staff',
 'Account Open Year',
 'age_of_regist_echannel_to_transaction',
 'Box Type Size',
 'Box Amount',
 'Is SDB Past Due',
 'Is TD Hold Amount',
 'Is Program Hold Amount',
 'Is Office Hour',
 'VALAS',
 'Last Transaction Datetime',
 'Last Transaction Amount',
 'Dr Last Transaction Date',
 'Dr Last Transaction Amount',
 'Cr Last Transaction Date',
 'Cr Last Transaction Amount',
 'Cr Total Transaction Amount Last 1 Day',
 'Cr Total Transaction Amount Last 7 Days',
 'Total Transaction Amount Last 1 Month?',
 'Dr Total Transaction Amount Last 1 Month',
 'Cr Total Transaction Amount Last 1 Month',
 'Average Transaction Amount Last 1 Day',
 'Average Transaction Amount Last 7 Days',
 'Average Transaction Amount Last 1 Month?',
 'Product Name',
 'Previous Account Status',
 'Remark 4 (Tabungan)',
 'HoldAmountTB',
 'Remark 4 (Deposito)',
 'Hold Amount DP',
 'Age of SDB Expiry Date Transaction Date',
 'SDB Product Code',
 'SDB Balance',